In [1]:
import numpy as np
import pandas as pd
from gensim.test.utils import datapath
from gensim import utils
import string
import re
import gensim.models
import tempfile
from gensim.models import Word2Vec
import time

# Training our model

In [2]:
filter_stopwords = False
filter_pronouns = False

# Reading in our sentences from the scrape results
matthew = False
path = "C:/Users/laure/OneDrive/Documenten/Project 3.1 APG Files/"
filename = 'public_and_private_sentences_filtered.csv'
if matthew:
    df = pd.read_csv('DATA/scrape_results/' + filename)
else:
    df = pd.read_csv(path + filename)


In [3]:
# Converting to list
corpus = df['sentences'].tolist()

In [4]:
# Break each sentence into list of individual words
sentences = []
for sentence in corpus:
    sentences.append(sentence.split(" "))

AttributeError: 'float' object has no attribute 'split'

In [ ]:
# Create custom word2vec model with given parameters (from internet)
custom_model = Word2Vec(window = 10, workers = 20, negative = 5, sample = 1e-5)

# Build the vocab by giving the sentences
custom_model.build_vocab(sentences, progress_per=10000)

# Train the model on the inputted sentences, epoch = 50, higher epochs give better output but increase time
start = time.time()
custom_model.train(sentences, total_examples=custom_model.corpus_count, epochs=50, report_delay=1)
end = time.time()

print("Time elapsed (seconds): " + str(end - start))

In [ ]:
# Save the model for re-use
filename = 'custom_model_sentences_filtered.wordvectors'
if matthew:
    custom_model.save('DATA/Data/' + filename)
else:
    custom_model.save(path + filename)

In [ ]:
# Some small tests
number_of_words = str(len(custom_model.wv))
print('Number of words: ' + number_of_words)

In [ ]:
# Test to see what words are similar to pensioen
most_similar_to_pensioen = custom_model.wv.most_similar(positive='pensioen')
print("Words most similar to pensioen: ", most_similar_to_pensioen)

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(path + 'models/model.bin', binary=True)

In [ ]:
print(model.similarity('pensioen', 'aow'))
model.most_similar(positive='pensioen')

### Comparing the Word2vec model trained on ABP's scrape results with the Word2vec model that is downloaded from [insert link].

In [ ]:
# Since the two models are created separately, the geometric positions of the embeddings in vector space are not directly comparable.
# To compare the embeddings, we first have to align the embeddings. The method 'align_embeddings' performs this alignment.


def align_embeddings(model1, model2):
    # Retrieving the common vocabulary.
    vocabulary1 = set(model1.wv.index_to_key)
    vocabulary2 = set(model2.index_to_key)
    common_vocabulary = vocabulary1 & vocabulary2

    # Extracting the word vectors from each model of the common vocabulary
    vectors1 = model1.wv[common_vocabulary]
    vectors2 = model2[common_vocabulary]

    # Computing and subtracting the average vectors.
    average1 = np.mean(vectors1, axis=0)
    average2 = np.mean(vectors2, axis=0)
    vectors1 = vectors1 - average1
    vectors2 = vectors2 - average2

    # Computing the matrix of dot products.
    dot_product = np.dot(vectors1.T, vectors2)

    # Using Singular Value Decomposition (SVD) to find the rotation matrix.
    U, s, Vt = np.linalg.svd(dot_product)

    # Using the rotation matrix to rotate the word vectors of 'model1' and re-averaging the word vectors to transform them back to the original space.
    rotated_vectors1 = np.dot(vectors2, U)
    rotated_vectors1 += average1

    # Using these rotated vectors, a new Word2Vec model can be created.
    aligned_model1 = Word2Vec(sentences=None, min_count=1)
    aligned_model1.build_vocab([common_vocabulary])
    aligned_model1.wv.vectors = rotated_vectors1
    #aligned_model1.train(common_vocabulary, total_examples=custom_model.corpus_count, epochs=50, report_delay=1)

    return aligned_model1

aligned_custom_model = align_embeddings(custom_model, model)






In [ ]:
most_similar_to_pensioen_aligned = aligned_custom_model.wv.most_similar(positive='pensioen', topn=100)
print("Words most similar to pensioen: ", most_similar_to_pensioen_aligned)

In [ ]:
aaaa = Word2Vec.load(path + "custom_model_none_filtered.bin")
aa = KeyedVectors.load_word2vec_format(path + "custom_model_none_filtered.bin", binary=True)